In [ ]:
import numpy as np
import cv2

def compute_disparity_dynamic(Il, Ir, sigma=2, c0=1):
    """Compute disparity map using dynamic programming.

    Args:
        Il: Left image (grayscale).
        Ir: Right image (grayscale).
        sigma: Pixel noise standard deviation.
        c0: Occlusion cost.

    Returns:
        Disparity map.
    """
    h, w = Il.shape
    disparity_map = np.zeros((h, w), dtype=np.float32)

    for row in range(h):
        left_row = Il[row, :]
        right_row = Ir[row, :]
        n = len(left_row)

        # Initialize cost matrix D
        D = np.zeros((n + 1, n + 1))
        D[0, :] = np.arange(n + 1) * c0
        D[:, 0] = np.arange(n + 1) * c0

        # Fill D
        for i in range(1, n + 1):
            for j in range(1, n + 1):
                dij = ((int(left_row[i - 1]) - int(right_row[j - 1])) ** 2) / (sigma ** 2)
                D[i, j] = min(
                    D[i - 1, j - 1] + dij,  # match
                    D[i - 1, j] + c0,       # skip left
                    D[i, j - 1] + c0        # skip right
                )

        # Backtracking to find disparities
        i, j = n, n
        disp_row = np.zeros(n)

        while i > 0 and j > 0:
            if D[i, j] == D[i-1, j-1] + ((int(left_row[i-1]) - int(right_row[j-1])) ** 2) / (sigma ** 2):
                disp_row[i-1] = abs(i - j)
                i -= 1
                j -= 1
            elif D[i, j] == D[i-1, j] + c0:
                disp_row[i-1] = 0  # left pixel unmatched
                i -= 1
            else:
                j -= 1  # right pixel unmatched

        disparity_map[row, :] = disp_row

    return disparity_map


In [ ]:

# Load stereo images
imgl = cv2.imread('stereo_materials/l2.png', cv2.IMREAD_GRAYSCALE)
imgr = cv2.imread('stereo_materials/r2.png', cv2.IMREAD_GRAYSCALE)

# Compute disparity map using dynamic programming
disparity_dp = compute_disparity_dynamic(imgl, imgr, sigma=2, c0=1)

# Normalize and show/save
disp_display = cv2.normalize(disparity_dp, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
disp_display = np.uint8(disp_display)

cv2.imshow('Disparity Dynamic Programming', disp_display)
cv2.imwrite('output/disparity_dynamic_programming.png', disp_display)

cv2.waitKey(0)
cv2.destroyAllWindows()
